## Helpful Links for Detectron2
- Guide to custom data training - https://www.analyticsvidhya.com/blog/2021/08/your-guide-to-object-detection-with-detectron2-in-pytorch/  
- Detectron2 configuration documentation - https://detectron2.readthedocs.io/en/latest/modules/config.html 
- Github link https://github.com/facebookresearch/detectron2 
- Another custom data training guide - https://towardsdatascience.com/train-maskrcnn-on-custom-dataset-with-detectron2-in-4-steps-5887a6aa135d


### Possible help for errors
- https://stackoverflow.com/questions/69002169/json-annotations-error-string-indices-must-be-integers
- https://stackoverflow.com/questions/63012735/typeerror-string-indices-must-be-integers-while-trying-to-train-mask-rcnn-imple


In [1]:
import json
from detectron2.data import MetadataCatalog, DatasetCatalog


def load_data(t="train"):
    if t == "train":
        with open("../data/detectron2/training/training.json", 'r') as file:
            train = json.load(file)
        return train
    elif t == "val":
      with open("../data/detectron2/validation/validation.json", 'r') as file:
          val = json.load(file)
    return val

In [2]:
from detectron2.config import get_cfg
from detectron2 import model_zoo
import os


def custom_config(num_classes):
    cfg = get_cfg()

    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

    cfg.MODEL.MASK_ON = True

    cfg.DATASETS.TRAIN = ("train",)
    cfg.DATASETS.TEST = ("val",)

    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.SOLVER.IMS_PER_BATCH = 4
    cfg.SOLVER.BASE_LR = 0.0002
    cfg.SOLVER.MAX_ITER = 20000

    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 32   # faster, enough for this dataset (default: 512)
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes + 1 # For background

    cfg.MODEL.DEVICE = 'cpu'
     
    cfg.OUTPUT_DIR = "mask_worms"

    cfg.TEST.DETECTIONS_PER_IMAGE = 300

    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    
    return cfg

In [3]:
for d in ["train", "val"]:
    DatasetCatalog.register(d, lambda d=d: load_data(d))
    MetadataCatalog.get(d).set(thing_classes=["background", "pbw", "abw"])

In [4]:
from detectron2.engine.hooks import HookBase
from detectron2.evaluation import inference_context
from detectron2.utils.logger import log_every_n_seconds
from detectron2.data import DatasetMapper, build_detection_test_loader
import detectron2.utils.comm as comm
import torch
import time
import datetime
import logging
import numpy as np

class LossEvalHook(HookBase):
    def __init__(self, eval_period, model, data_loader):
        self._model = model
        self._period = eval_period
        self._data_loader = data_loader
    
    def _do_loss_eval(self):
        # Copying inference_on_dataset from evaluator.py
        total = len(self._data_loader)
        num_warmup = min(5, total - 1)
            
        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []
        for idx, inputs in enumerate(self._data_loader):            
            if idx == num_warmup:
                start_time = time.perf_counter()
                total_compute_time = 0
            start_compute_time = time.perf_counter()
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            total_compute_time += time.perf_counter() - start_compute_time
            iters_after_start = idx + 1 - num_warmup * int(idx >= num_warmup)
            seconds_per_img = total_compute_time / iters_after_start
            if idx >= num_warmup * 2 or seconds_per_img > 5:
                total_seconds_per_img = (time.perf_counter() - start_time) / iters_after_start
                eta = datetime.timedelta(seconds=int(total_seconds_per_img * (total - idx - 1)))
                log_every_n_seconds(
                    logging.INFO,
                    "Loss on Validation  done {}/{}. {:.4f} s / img. ETA={}".format(
                        idx + 1, total, seconds_per_img, str(eta)
                    ),
                    n=5,
                )
            loss_batch = self._get_loss(inputs)
            losses.append(loss_batch)
        mean_loss = np.mean(losses)
        self.trainer.storage.put_scalar('validation_loss', mean_loss)
        comm.synchronize()

        return losses
            
    def _get_loss(self, data):
        # How loss is calculated on train_loop 
        metrics_dict = self._model(data)
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }
        total_losses_reduced = sum(loss for loss in metrics_dict.values())
        return total_losses_reduced
        
        
    def after_step(self):
        next_iter = self.trainer.iter + 1
        is_final = next_iter == self.trainer.max_iter
        if is_final or (self._period > 0 and next_iter % self._period == 0):
            self._do_loss_eval()
        self.trainer.storage.put_scalars(timetest=12)

In [5]:
from detectron2.data import DatasetMapper, build_detection_test_loader
from detectron2.engine import DefaultTrainer

class CustomTrainer(DefaultTrainer):
    """
    Custom Trainer deriving from the "DefaultTrainer"

    Overloads build_hooks to add a hook to calculate loss on the test set during training.
    """

    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(-1, LossEvalHook(
            1000, # Frequency of calculation - every 100 iterations here
            self.model,
            build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                DatasetMapper(self.cfg, True)
            )
        ))

        return hooks

In [6]:
from detectron2.data import MetadataCatalog, DatasetCatalog
    
metadata = MetadataCatalog.get("train")

cfg = custom_config(2)

trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[11/27 14:11:17 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (12, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (3, 256, 1, 1) 

[11/27 14:11:18 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/Caskroom/miniforge/base/envs/detectron2/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1666646603923/work/aten/src/ATen/native/TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[11/27 14:13:38 d2.utils.events]:  eta: 1 day, 14:46:09  iter: 19  total_loss: 3.179  loss_cls: 1.224  loss_box_reg: 0.7707  loss_mask: 0.6983  loss_rpn_cls: 0.4234  loss_rpn_loc: 0.07801  time: 6.8281  data_time: 0.2436  lr: 3.9962e-06  
[11/27 14:15:55 d2.utils.events]:  eta: 1 day, 14:50:59  iter: 39  total_loss: 3.196  loss_cls: 1.155  loss_box_reg: 0.7663  loss_mask: 0.6945  loss_rpn_cls: 0.6063  loss_rpn_loc: 0.08071  time: 6.8392  data_time: 0.1444  lr: 7.9922e-06  
[11/27 14:18:08 d2.utils.events]:  eta: 1 day, 13:44:17  iter: 59  total_loss: 2.865  loss_cls: 1.048  loss_box_reg: 0.7954  loss_mask: 0.6848  loss_rpn_cls: 0.1775  loss_rpn_loc: 0.103  time: 6.7648  data_time: 0.1256  lr: 1.1988e-05  
[11/27 14:20:21 d2.utils.events]:  eta: 1 day, 13:17:12  iter: 79  total_loss: 2.613  loss_cls: 0.9235  loss_box_reg: 0.7992  loss_mask: 0.6724  loss_rpn_cls: 0.08383  loss_rpn_loc: 0.05578  time: 6.7463  data_time: 0.1496  lr: 1.5984e-05  
[11/27 14:22:37 d2.utils.events]:  eta: 1 da

In [7]:
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer, ColorMode
import matplotlib.pyplot as plt
import cv2
import os


def test_and_visualize(metadata, cfg, test_set, model_file):
    test_results = []
    
    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, model_file)
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3

    predictor = DefaultPredictor(cfg)
    for d in test_set:
        im = cv2.imread(d["file_name"])
        outputs = predictor(im)
        v = Visualizer(im[:, :, ::-1],
                       metadata=metadata,
                       scale=0.5,
                       instance_mode=ColorMode.IMAGE_BW
                       )
        instances = outputs["instances"].to("cpu")
        test_results.append({
            "file_name": d["file_name"],
            "pred_classes": instances.pred_classes.detach().numpy(),
            "scores": instances.scores.detach().numpy()
        })
        out = v.draw_instance_predictions(instances)
        img = cv2.cvtColor(out.get_image()[:, :, ::-1], cv2.COLOR_RGBA2RGB)
        plt.imsave(os.path.join(os.path.join(cfg.OUTPUT_DIR, 'visualizations'), str(d["image_id"]) + '.jpg'), img)

    return test_results

In [8]:
import numpy as np

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

for itrn in np.arange(4999, 19999, 5000):
    with open('../data/detectron2/test.json') as f:
        test_set = json.loads(f.read())
        results = test_and_visualize(metadata, cfg, test_set, f"model_{str.rjust(str(itrn), 7, '0')}.pth")

    test_results_json = json.dumps(results, cls=NpEncoder)

    with open(f'../results/detectron2/test_results_{str(itrn)}.json', 'w') as f:
        f.write(test_results_json)

# Run the final iteration
with open('../data/detectron2/test.json') as f:
    test_set = json.loads(f.read())
    results = test_and_visualize(metadata, cfg, test_set, f"model_final.pth")

test_results_json = json.dumps(results, cls=NpEncoder)

with open(f'../results/detectron2/test_results_final.json', 'w') as f:
    f.write(test_results_json)


[11/29 07:23:16 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                                  | Names in Checkpoint                                                                                  | Shapes                                          |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------|:------------------------------------------------|
| backbone.bottom_up.res2.0.conv1.*               | backbone.bottom_up.res2.0.conv1.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| backbone.bottom_up.res2.0.conv2.*               | backbone.bottom_up.res2.0.conv2.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| backbone.bottom_up.res2.0.conv3.*               | backbone.bottom_up.res2.0.conv3.{norm.bia

Invalid SOS parameters for sequential JPEG


[11/29 08:44:40 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                                  | Names in Checkpoint                                                                                  | Shapes                                          |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------|:------------------------------------------------|
| backbone.bottom_up.res2.0.conv1.*               | backbone.bottom_up.res2.0.conv1.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| backbone.bottom_up.res2.0.conv2.*               | backbone.bottom_up.res2.0.conv2.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| backbone.bottom_up.res2.0.conv3.*               | backbone.bottom_up.res2.0.conv3.{norm.bia

Invalid SOS parameters for sequential JPEG


[11/29 10:07:30 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                                  | Names in Checkpoint                                                                                  | Shapes                                          |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------|:------------------------------------------------|
| backbone.bottom_up.res2.0.conv1.*               | backbone.bottom_up.res2.0.conv1.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| backbone.bottom_up.res2.0.conv2.*               | backbone.bottom_up.res2.0.conv2.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| backbone.bottom_up.res2.0.conv3.*               | backbone.bottom_up.res2.0.conv3.{norm.bia

Invalid SOS parameters for sequential JPEG


[11/29 11:29:39 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                                  | Names in Checkpoint                                                                                  | Shapes                                          |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------|:------------------------------------------------|
| backbone.bottom_up.res2.0.conv1.*               | backbone.bottom_up.res2.0.conv1.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| backbone.bottom_up.res2.0.conv2.*               | backbone.bottom_up.res2.0.conv2.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| backbone.bottom_up.res2.0.conv3.*               | backbone.bottom_up.res2.0.conv3.{norm.bia

Invalid SOS parameters for sequential JPEG


In [9]:
import pandas as pd

test_df = pd.read_csv('../data/test.csv')

abw_df = test_df.copy(deep=True)
abw_df['worm_type'] = 'abw'
pbw_df = test_df.copy(deep=True)
pbw_df['worm_type'] = 'pbw'

full_test_df = pd.concat([abw_df, pbw_df], ignore_index=True)
full_test_df['image_id_worm'] = full_test_df['image_id_worm'].apply(lambda x: os.path.splitext(x)[0])
full_test_df['image_id_worm'] = (full_test_df['image_id_worm'] + "_" + full_test_df['worm_type']).str.lower()
full_test_df = full_test_df.drop(columns=['worm_type'])
full_test_df = full_test_df.rename(columns={"image_id_worm": "Image_ID"})
full_test_df

,Image_ID
0,id_00332970f80fa9a47a39516d_abw
1,id_0035981bc3ae42eb5b57a317_abw
2,id_005102f664b820f778291dee_abw
3,id_0066456f5fb2cd858c69ab39_abw
4,id_007159c1fa015ba6f394deeb_abw
...,...
5601,id_ffad8f3773a4222f8fe5ba1a_pbw
5602,id_ffb65e6de900c49d8f2ef95a_pbw
5603,id_ffbcb27fa549278f47505515_pbw
5604,id_ffc0e41e10b0c964d4a02811_pbw


In [10]:
for threshold in np.arange(0.3, 0.7, 0.1):
    for itrn in [*np.arange(4999, 19999, 5000), 'final']:
        test_results_df = pd.read_json(f'../results/detectron2/test_results_{itrn}.json')
        test_results_df['file_name'] = test_results_df['file_name'].apply(lambda x: x.replace('../data/images/', ''))
        test_results_df = test_results_df.explode(['pred_classes', 'scores'])
        test_results_df = test_results_df[test_results_df['scores'] > threshold]
        test_results_df = test_results_df.drop(columns=['scores'])
        test_results_df = test_results_df.value_counts().rename_axis(['image_id', 'worm_type']).reset_index(name='target')
        test_results_df['worm_type'] = test_results_df['worm_type'].apply(lambda x: 'abw' if x == 1 else 'pbw')
        test_results_df['image_id'] = test_results_df['image_id'].apply(lambda x: x.split('.')[0])

        final_testing_df = test_results_df.copy(deep=True)

        final_testing_df['Image_ID'] = final_testing_df['image_id']
        final_testing_df['Image_ID'] = final_testing_df['Image_ID'].apply(lambda x: os.path.splitext(x)[0])
        final_testing_df['Image_ID'] = (final_testing_df['Image_ID'] + "_" + final_testing_df['worm_type']).str.lower()
        final_testing_df = pd.merge(full_test_df, final_testing_df, on="Image_ID", how="left")

        final_testing_df = final_testing_df.drop(columns=['image_id', 'worm_type'])
        final_testing_df['target'] = final_testing_df['target'].fillna(0)
        final_testing_df['target'] = pd.to_numeric(final_testing_df['target'], downcast='integer')
        final_testing_df = final_testing_df.rename(columns={"Target": "target"})
        final_testing_df = final_testing_df.sort_values(by='Image_ID').reset_index(drop=True)

        final_testing_df.to_csv(f'../results/detectron2/detectron2_test_results_{itrn}_{str(round(threshold, 1))}.csv', index=False)
        final_testing_df